## Part 2 Molecular adsorption

Supervisor: Prof. Aschauer

Assistants: Dr. Xing Wang

Room:	N216

### 1. Clean La$_2$Ti$_2$O$_7$ surface

ASE provides a `surface` function to setup the surface. Here we focus on the (010) surface of La$_2$Ti$_2$O$_7$. In the atomic structure, we kept the bottom two atomic layers fixed at their bulk positions, which is an (admittedly pretty drastic) approximation of the surface of a very large solid.


We should build a symmetric and stoichiometric surface slab if it is possible. Please discuss with me when you doing this.

Here (010) surface:



In [28]:
from ase.io import read
from ase.build import surface, add_adsorbate
from ase.constraints import FixAtoms
from ase.geometry import get_layers
from ase.visualize import view
from xespresso import Espresso
from xespresso.tools import mypool, fix_layers, dipole_correction
from x3dase.visualize import view_x3d_n
# read the optmized La2Ti2O7 structure
lto = read('datas/La2Ti2O7.cif')
# lto = Espresso(label = 'bulk/lto').results['atoms']
slab = surface(lto, (0, 1, 0), 4, vacuum = 5.0)
slab.pbc = [True, True, True]
# view(slab)
layers = get_layers(slab, (0, 0, 1), 0.5)[0]
index = [j for j in range(len(slab)) if layers[j] in range(2, 11)]
slab = slab[index]
# view(slab)
del slab[[81, 90, 92, 91, 13, 16, 15, 24, 85, 12]]
# view(slab)
slab.positions[:, 2] -= min(slab.positions[:, 2]) - 0.01
slab.positions[:, 1] -= slab.cell[1][1]/2.0
slab.wrap()
slab.cell[2][2] = max(slab.positions[:, 2]) + 15
slab = fix_layers(slab, (0, 0, 1), 0.5, [0, 2])
print(slab)
view_x3d_n(slab, bond = 1.0, label = True, output = 'htmls/lto-surface.html')

Atoms(symbols='La16O56Ti16', pbc=True, cell=[[13.219083400000002, 0.0, 0.0], [-1.1597661595305675, 7.7306691565224614, 0.0], [0.0, -0.0, 26.500206001183003]], spacegroup_kinds=..., constraint=FixAtoms(indices=[9, 11, 14, 16, 19]))


Then build the Espresso calculator.

In [ ]:
# For this big bulk system, we use the "empi" partition
queue = {'nodes': 4, 'ntasks-per-node': 20, 'partition': 'empi', 'mem-per-cpu': '5G', 'time': '23:59:00'}

input_ntyp = {
    'Hubbard_U': {'Ti': 3.0},
    'starting_magnetization': {'Ti': 0.0}}

calc = Espresso(label = 'surface/lto-010',
                pseudopotentials = mypsudo,
                calculation = 'relax',   # allow atoms to move
                ecutwfc = 40,
                occupations = 'smearing',
                degauss = 0.01,
                kpts = (2, 4, 1),        # k-points, 
                electron_maxstep = 200,
                mixing_mode = 'local-TF',
                nspin = 2,
                lda_plus_u = True,
                input_data = {'input_ntyp': input_ntyp},
                queue = queue,
                parallel = '-nk 5',)
slab.calc = calc
atoms.get_potential_energy()
e = slab.calc.results['energy']
print('Energy {0:3.3f}'.format(e))

### 2. Molecular adsorption
The La$_2$Ti$_2$O$_7$ surface exposes four undercoordinated La and four undercoordinated Ti atoms, surrounded by O atoms, that represent adsorption sites for the NH$_3$. Molecular NH$_3$ will adsorb on the surface in a lots of geometries. We will try the configurations with the N atoms about 1Å above the follwoing surface sites:

* at the top site (La, Ti)
* at the bridge site between La and Ti atoms

Let's look at the La$_2$Ti$_2$O$_7$ surface again and chose approximate positions for the adsorption sites. Then add those position information to the `adsorbate_info` of the slab.

In [29]:
# set adsorption site on the slab
slab_opt = slab.calc.results['atoms']
positions = slab.get_scaled_positions()
sites = {
    'Ti-top': positions[74][0:2], # the (x, y) position of the atom 51 in the slab
    # 'O-top': positions[75][0:2], 
    'La-top': positions[67][0:2], 
    'Ti-La-bridge': (positions[67][0:2] + positions[73][0:2])/2.0,  # bridge site between atom 67 and atom 73
    }
slab_opt.info['adsorbate_info'] = {'sites': sites}

Is there other posible adsorption sites and geometries?

Now we add NH$_3$ on the adsorption sites. Please check all configurations.

In [36]:
from ase.build import molecule, add_adsorbate
nh3 = molecule('NH3')
nh3.rotate(180, 'x') # rotate the molecule to make N atoms in the downside
jobs = {}
for site in slab.info['adsorbate_info']['sites']:
    print(site)
    atoms = slab_opt.copy()
    add_adsorbate(atoms, nh3, 1.0, site, mol_index=0)
    jobs[site] = atoms
# We can check the structure 
view(jobs.values())
site = 'Ti-top'
view_x3d_n(jobs[site], bond = 1.0, label = True, output = 'htmls/%s-N.html'%site)

Ti-top
O-top
La-top
Ti-La-bridge



Each calculation will take a long time, therefore we will not run the calculation one by one. Instead, a `mypool` function will be used to sumbit all the calculations at the same time, thus all jobs will run parallelly.

In [15]:
from xespresso import Espresso
from xespresso.tools import mypool
from pseudo import mypseudo


queue = {'nodes': 4, 'ntasks-per-node': 20, 'partition': 'empi', 'mem-per-cpu': '5G', 'time': '23:59:00'}

molecular_energies = {}
# def a function to run a calculation for one site
def run(job, atoms):
    calc = Espresso(label = 'surface/nh3/%s'%job,
                    pseudopotentials = mypsudo,
                    calculation = 'relax',   # allow atoms to move
                    ecutwfc = 40,
                    occupations = 'smearing',
                    degauss = 0.01,
                    kpts = (2, 4, 1),        # k-points, 
                    electron_maxstep = 200,
                    mixing_mode = 'local-TF',
                    nspin = 2,
                    lda_plus_u = True,
                    input_data = {'input_ntyp': input_ntyp},
                    queue = queue,
                    parallel = '-nk 5',)
    atoms.calc = calc
    atoms.get_potential_energy()
    e = atoms.calc.results['energy']
    print('{0} {1:3.3f}'.format(site, e))
    return job, calc
# submit the calculation for all site at the same time
mypool(jobs, run)

Same geometry and parameters, try to check the results are available or not.
top-top -11543.680
Same geometry and parameters, try to check the results are available or not.
top-fcc -11543.638


### 4. Calculation the adsorption energy
The adsorption energy is given by

$\Delta E_{ads} = E_{surf + ads} - (E_{surf} + E_{ads})$

where $E_{surf + ads}$ is the energy of the surface with the adsorbate (NH$_3$), $E_{surf}$ is the energy of the clean surface and $E_{ads}$ is the energy of the adsorbate. A negative value means that there is an energy gain for adsorption, while a positive value means that adsorption is not energetically favorable.

In [40]:
#=====================================================
#  Molecular adsorption
E_nh3 = ?
E_lto = ?
molecular_energies = {
'Ti-top': ?,
'O-top': ?,
'La-top': ?,
?
?
}
print('Sites     Adsorption energies (eV)')
for site, E_ads in molecular_energies.items():
    dE_ads = E_ads - (E_lto + E_nh3)
    print('{0:10s}  {1:1.2f}'.format(site, dE_ads))

Sites     Adsorption energies (eV)
top-top     -0.74
top-fcc     -0.70
